In [6]:
import os
import utils
import numpy as np
import ray


how should we use this system

1. we need a quick way to iterate thru a folder or a single file and return available stats names
   this will be the mega list and user will delete which ever ones they dont need and save the 
   final condensed list as a .txt file. if multiple files or surfaces require different stats 
   they can create multiple .txt files and then pass them in seperately. 

2. we have the assumption that for each folder the user will create a seperate config file which
   is the meta_config file. 

In [ ]:
# how should the config file look for a single ims file
# this config should be constructed for each file based on meta_config file

# config = {
    
#     'ims_file_path': ims_file_path,
#     'csv_save_path': csv_save_path,
#     'valid_surfaces': {
#         'surface_1': {
#             'valid_stats': valid_stats,
#             'organization_order': organization_order,
#         },
#         'surface_2': {             
#             'valid_stats': valid_stats,
#             'organization_order': organization_order,
#         }
#     },
#     available_stats_save_path: available_stats_save_path,
# }

In [ ]:
# how should the meta_config file look like



In [8]:
class SurfaceDataContainer:
    """
    Usage Notes:
    
    -- a single instance of the class will parse stats for a single/multiple surfaces
    -- the surface number(s) for which stats needs to be calculated must be includes in config
    -- the final surface that will be generated will be saved in the same folder(ask about this)?
    -- this function is only used to neatly contain information of a single ims file
    """
    
    def __init__(self, 
                 ims_file_path: str,
                 csv_save_path: str,
                 valid_surface: int,
                 valid_stats: list,
                 organization_order: list, 
                 available_stats_save_path: str=None):
        
        # load the data so the class has access to this information
        self.data = utils.load_ims(ims_file_path)
        
        # based on the data get the available metasurfaces
        self.surface = utils.get_object_names(
            full_data_file = self.data, 
            search_for = 'Surface')[valid_surface]
        
        # get names of all available statistics names
        # format -> key=numeric_id, value=name
        self.available_stats = utils.get_statistics_names(
            full_data_file = self.data,
            object_name = self.surface,
        )
        
        # invert and reorganize stats and flatten
        # format -> key=name, value=numeric_id
        # the version saved to user to edit
        self.inverted_dict = utils.invert_stats_dict(self.available_stats)
        self.inverted_dict = utils.flatten(self.inverted_dict)
        
        # reverse flatten dict
        # format -> key=numeric_id, value=name
        self.available_stats_final = {v: k for k,v in self.inverted_dict.items()}
        
        # generate a filename
        # filename = ims_file_path + valid_surface
        self.filename = ims_file_path + f'_{self.surface}'
        
        # if available stats save path is provided 
        # we dont extract and data, we simply provide the user with the available stats
        if available_stats_save_path:
            save_path = os.path.join(available_stats_save_path, f'{self.filename}.txt')
            np.savetxt(save_path, list(self.inverted_dict.keys()), fmt='%s')

In [ ]:
# def extract_and_save(data, surface_name, avilable_stats, )

In [ ]:
    # def filter_statistics(self, inverted_flatten: dict, categories: list) -> list:
    #     """
    #     filter_statistics will search the inverted_flatten dict where data is
    #     organized as key=name, value=numeric id to identify statistics we are
    #     not interested in. The statistics name we are interested in will be 
    #     given in the categories list. 

    #     Args:
    #         inverted_flatten (dict): a dictionary with key=stats name, value=numeric id
    #         categories (list): a list of statistics names we are interested in

    #     Returns:
    #         list: a list of statistics name we want to filter out. 
    #     """

    #     # create a empty list to store data we want to delete
    #     del_list = [None] * (len(inverted_flatten) - len(categories))
    #     # iterate and filter
    #     counter = 0
    #     for stats_name, num_id in inverted_flatten.items():
    #         if stats_name not in categories and num_id != None:
    #             del_list[counter] = num_id
    #             counter += 1
        
    #     return del_list